# Masking data

In [39]:
import io
import os
import json
import pandas as pd
import numpy as np
import random
import copy
import distutils.dir_util
from matplotlib import pyplot as plt

In [40]:
# json write & load 함수 정의
def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname)
    distutils.dir_util.mkpath(parent)
    with io.open(fname, "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)
        
def load_json(fname):
    with open(fname, encoding='utf-8') as f:
        json_obj = json.load(f)

    return json_obj

In [42]:
test = load_json('../0_data/test.json')

In [43]:
playlists = copy.deepcopy(test)
tot = len(playlists)
tot

11456

In [44]:
song_only = playlists[:int(tot * 0.3)] # 0.3
song_and_tags = playlists[int(tot * 0.3):int(tot * 0.8)] # 0.5
tags_only = playlists[int(tot * 0.8):int(tot * 0.95)] # 0.15
title_only = playlists[int(tot * 0.95):] # 0.05

print(f"Total: {len(playlists)}, "
        f"Song only: {len(song_only)}, "
        f"Song & Tags: {len(song_and_tags)}, "
        f"Tags only: {len(tags_only)}, "
        f"Title only: {len(title_only)}")

Total: 11456, Song only: 3436, Song & Tags: 5728, Tags only: 1719, Title only: 573


In [45]:
def mask(playlists, mask_cols, del_cols):
    q_pl = copy.deepcopy(playlists)
    a_pl = copy.deepcopy(playlists)

    for i in range(len(playlists)):
        for del_col in del_cols:
            q_pl[i][del_col] = []
            if del_col == 'songs':
                a_pl[i][del_col] = a_pl[i][del_col][:100]
            elif del_col == 'tags':
                a_pl[i][del_col] = a_pl[i][del_col][:10]

        for col in mask_cols:
            mask_len = len(playlists[i][col])
            mask = np.full(mask_len, False) # mask_len 만큼 False로 채워진 행렬 생성
            mask[:mask_len//2] = True # mask_len의 절반을 True로 바꿈
            np.random.shuffle(mask) 

            q_pl[i][col] = list(np.array(q_pl[i][col])[mask])
            a_pl[i][col] = list(np.array(a_pl[i][col])[np.invert(mask)])
            
    return q_pl, a_pl

In [46]:
song_q, song_a = mask(song_only, ['songs'], ['tags'])
songtag_q, songtag_a = mask(song_and_tags, ['songs', 'tags'], [])
tag_q, tag_a = mask(tags_only, ['tags'], ['songs'])
title_q, title_a = mask(title_only, [], ['songs', 'tags'])

q = song_q + songtag_q + tag_q + title_q
a = song_a + songtag_a + tag_a + title_a

shuffle_indices = np.arange(len(q))
np.random.shuffle(shuffle_indices)

q = list(np.array(q)[shuffle_indices])
a = list(np.array(a)[shuffle_indices])

In [47]:
write_json(q, "../0_data/q_test.json")
write_json(a, "../0_data/a_test.json")

In [48]:
mt = pd.read_json('../0_data/q_test.json')
at = pd.read_json('../0_data/a_test.json')
t = pd.read_json('../0_data/test.json')

In [34]:
mt.head(20)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[소울],81539,너무나 감성을 자극하는 너무나 감각있는,[],12,2013-11-02 09:13:04.000
1,[],4526,오늘 한강 피크닉 어떠세요,"[467385, 255495, 368546, 667154, 326544]",33,2020-02-11 18:54:12.000
2,[],2333,유니크한 일렉 & 힙합 뮤직,"[187482, 203189, 491365, 587167, 140041, 39910...",65,2016-02-24 23:02:18.000
3,"[카페, 힐링, 공부]",17795,카페에서 이어폰 꽂고 공부&책 읽으면서 듣는 가사 없는 음악,"[480158, 311206, 198701, 695554, 462255, 33639...",28,2017-03-17 15:15:33.000
4,[],57973,아무생각없이 즐기는 노래,"[430159, 543751, 315736, 591047, 155480, 64473...",6,2015-08-27 16:21:30.000
5,"[매장음악, 뉴에이지, 청량감]",107390,청량감 넘치는 Summer Classical BGM,"[10991, 238465]",14,2016-08-11 16:31:50.000
6,"[감성, 가을, 잔잔한, 도서관]",142857,주변소음 안녕 독서할때 공부할때 들으면 딱인 노래들,[575964],7,2017-09-27 17:47:14.000
7,"[감성, 그루브]",94358,나만 알고싶은 그루브하고 촉촉한 감성 팝,"[219225, 427835, 603681, 345649, 494471, 37406...",2811,2017-04-24 12:33:50.000
8,[],57819,퓨전 재즈 팝재즈,"[464984, 609952, 520983, 491278]",5,2016-03-20 16:54:30.000
9,[],50767,믿고 틀어두시면 됩니다19 20 연말 드라이빙 음악 mix 베스트100곡,"[219849, 61757, 18647, 117953, 376749, 137754,...",11,2019-12-25 17:01:45.000


In [35]:
at.head(20)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[알앤비],81539,너무나 감성을 자극하는 너무나 감각있는,[241207],12,2013-11-02 09:13:04.000
1,"[기분좋은, 피크닉, 한강, 가을, 소풍, 공원, 2000년대, 인디, 일탈]",4526,오늘 한강 피크닉 어떠세요,"[588746, 407219, 421743, 365824, 29114, 696302]",33,2020-02-11 18:54:12.000
2,[일렉],2333,유니크한 일렉 & 힙합 뮤직,"[651120, 222602, 378589, 101182, 228740, 34258...",65,2016-02-24 23:02:18.000
3,"[집중력, 집중, 독서, 편안한]",17795,카페에서 이어폰 꽂고 공부&책 읽으면서 듣는 가사 없는 음악,"[145369, 567112, 103086, 524356, 654478, 49359...",28,2017-03-17 15:15:33.000
4,"[기분전환, 운동]",57973,아무생각없이 즐기는 노래,"[555134, 656431, 707837, 98472, 397412, 626581...",6,2015-08-27 16:21:30.000
5,"[여름, 카페, bgm]",107390,청량감 넘치는 Summer Classical BGM,"[325852, 668471]",14,2016-08-11 16:31:50.000
6,"[매장음악, 카페, 독서, 공부]",142857,주변소음 안녕 독서할때 공부할때 들으면 딱인 노래들,[494862],7,2017-09-27 17:47:14.000
7,"[귀르가즘, 팝, 유니크]",94358,나만 알고싶은 그루브하고 촉촉한 감성 팝,"[387189, 462080, 226137, 430821, 695407, 47891...",2811,2017-04-24 12:33:50.000
8,[퓨전재즈],57819,퓨전 재즈 팝재즈,"[499897, 150336, 173453, 287137, 366567]",5,2016-03-20 16:54:30.000
9,"[연말, 연인, 분위기, 겨울, 스펙트럼, 크리스마스, 캐롤, 드라이빙]",50767,믿고 틀어두시면 됩니다19 20 연말 드라이빙 음악 mix 베스트100곡,"[348215, 408035, 60622, 545301, 263435, 401226...",11,2019-12-25 17:01:45.000


In [38]:
t[t['id']==4526]

,tags,id,plylst_title,songs,like_cnt,updt_date
5673,"[기분좋은, 피크닉, 한강, 가을, 소풍, 공원, 2000년대, 인디, 일탈]",4526,오늘 한강 피크닉 어떠세요,"[467385, 588746, 255495, 407219, 368546, 42174...",33,2020-02-11 18:54:12.000


In [32]:
len(mt), len(a), len(t)

(45824, 45824, 45824)

In [ ]:
# train: orig/train / predict: question/val / 평가: answer/val
# input: mask_train , output: train / predict: test / 평가 test 